In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import gensim

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import strip_accents_unicode

import re

import nltk
from nltk.tokenize import RegexpTokenizer

from gsdmm.gsdmm import MovieGroupProcess

In [2]:
def holdout_text(series,train_pct = 0.9):
    '''Input : Series de dados
       Output : train,test a partir da serie de dados completamente aleatoria'''

    if(type(series) == list):
        series = pd.Series(series)
        
    num_linhas = len(series)
    series = series.sample(num_linhas)#.reset_index(drop = True)

    train_pct = train_pct

    train = series.iloc[:round(num_linhas*train_pct)]
    test = series.iloc[round(num_linhas*train_pct):]
    
    return train,test

#### Carregando Dataset Americanas

In [3]:
df = pd.read_csv("B2W-Reviews01.csv",sep = ';')

#dropping IDs
df.drop(['product_id'],axis=1,inplace = True,errors = 'ignore') #PROD ID
df.drop(['reviewer_id '],axis=1,inplace = True,errors = 'ignore') # REVIEWERID

df.info()

/home/saraiva/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132373 entries, 0 to 132372
Data columns (total 13 columns):
submission_date          132373 non-null object
reviewer_id              132373 non-null object
product_name             132289 non-null object
product_brand            40982 non-null object
site_category_lv1        132367 non-null object
site_category_lv2        128360 non-null object
review_title             132373 non-null object
overall_rating           132373 non-null int64
recommend_to_a_friend    132355 non-null object
review_text              132373 non-null object
reviewer_birth_year      126389 non-null float64
reviewer_gender          128237 non-null object
reviewer_state           128382 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 13.1+ MB


In [13]:
df.site_category_lv1.value_counts()

Celulares e Smartphones           20859
Eletroportáteis                   11745
Beleza e Perfumaria                8962
Utilidades Domésticas              8193
TV e Home Theater                  7561
Informática e Acessórios           7469
Móveis                             7166
Brinquedos                         4712
Casa e Construção                  4467
Informática                        4262
Livros                             4203
Eletrodomésticos                   3905
Automotivo                         3879
Games                              3514
Bebês                              3011
Cama, Mesa e Banho                 2974
Esporte e Lazer                    2954
Saúde                              2761
Moda                               2504
Ferramentas e Jardim               1741
Relógios                           1496
Áudio                              1496
Decoração                          1467
Pet Shop                           1341
Suplementos e Vitaminas            1215


In [4]:
#dados de texto
text_data = df['review_text']
#text_title = df['review_title']

#stopwords
spwrds = nltk.corpus.stopwords.words('portuguese')

pp_text = [strip_accents_unicode(phrases) for phrases in text_data]
pp_text = [phrases.lower() for phrases in pp_text]

#regex_numbers 
pp_text = [re.sub(r'\d+','<NUM>', phrases) for phrases in pp_text]

#strip para ter certeza de espacos
pp_text = [phrases.strip() for phrases in pp_text]


#rgxtoken = RegexpTokenizer('\w+')
#pp_text = [rgxtoken.tokenize(phrases) for phrases in pp_text]

In [5]:
train,test =  holdout_text(pp_text,train_pct=0.7)

In [6]:
%%time
tfidf = TfidfVectorizer(stop_words = spwrds,min_df = 10)
tfidf.fit(train)

#train = tfidf.transform(train)
#test = tfidf.transform(test)

CPU times: user 2.38 s, sys: 23.8 ms, total: 2.4 s
Wall time: 2.79 s


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=10, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em',
                            'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se',
                            'na', 'por', 'mais', 'as', 'dos', 'como', 'mas',
                            'ao', 'ele', 'das', 'à', 'seu', 'sua', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [7]:
mgp = MovieGroupProcess(K=100, alpha=0.1, beta=0.5, n_iters=20)

mgp.fit([[elements] for elements in train][:1000],
        len(tfidf.vocabulary_.keys()))

In stage 0: transferred 987 clusters with 98 clusters populated
In stage 1: transferred 989 clusters with 93 clusters populated
In stage 2: transferred 987 clusters with 83 clusters populated
In stage 3: transferred 978 clusters with 76 clusters populated
In stage 4: transferred 982 clusters with 71 clusters populated
In stage 5: transferred 975 clusters with 68 clusters populated
In stage 6: transferred 978 clusters with 63 clusters populated
In stage 7: transferred 966 clusters with 58 clusters populated
In stage 8: transferred 967 clusters with 51 clusters populated
In stage 9: transferred 970 clusters with 51 clusters populated
In stage 10: transferred 971 clusters with 48 clusters populated
In stage 11: transferred 969 clusters with 48 clusters populated
In stage 12: transferred 967 clusters with 45 clusters populated
In stage 13: transferred 948 clusters with 46 clusters populated
In stage 14: transferred 951 clusters with 46 clusters populated
In stage 15: transferred 943 cluste

[48,
 15,
 46,
 56,
 46,
 76,
 79,
 71,
 93,
 59,
 9,
 60,
 59,
 1,
 79,
 59,
 26,
 1,
 71,
 15,
 8,
 46,
 10,
 76,
 51,
 79,
 71,
 52,
 28,
 71,
 51,
 51,
 26,
 71,
 17,
 8,
 71,
 91,
 14,
 9,
 46,
 46,
 56,
 33,
 52,
 48,
 56,
 14,
 52,
 79,
 1,
 71,
 19,
 48,
 56,
 88,
 51,
 52,
 33,
 28,
 79,
 52,
 59,
 52,
 41,
 56,
 51,
 48,
 79,
 26,
 41,
 59,
 41,
 26,
 56,
 15,
 8,
 8,
 56,
 5,
 99,
 59,
 57,
 57,
 14,
 41,
 1,
 88,
 71,
 5,
 47,
 71,
 59,
 56,
 52,
 48,
 52,
 51,
 52,
 88,
 52,
 79,
 10,
 8,
 59,
 58,
 46,
 76,
 61,
 71,
 8,
 1,
 61,
 88,
 79,
 88,
 79,
 14,
 76,
 71,
 57,
 5,
 19,
 10,
 88,
 76,
 56,
 71,
 8,
 51,
 3,
 71,
 1,
 64,
 15,
 33,
 56,
 91,
 28,
 91,
 46,
 28,
 10,
 3,
 51,
 3,
 46,
 76,
 79,
 80,
 71,
 71,
 57,
 91,
 88,
 79,
 71,
 79,
 8,
 3,
 52,
 71,
 52,
 41,
 52,
 9,
 41,
 71,
 71,
 8,
 41,
 71,
 48,
 56,
 57,
 60,
 8,
 17,
 33,
 59,
 88,
 5,
 3,
 8,
 14,
 79,
 52,
 71,
 51,
 88,
 52,
 71,
 71,
 51,
 76,
 3,
 56,
 33,
 26,
 76,
 8,
 10,
 52,
 52,
 88,
 48,
 

In [10]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)
print('*'*20)
# Topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-10:][::-1]
print('Most important clusters (by number of docs inside):', top_index)
print('*'*20)
# Show the top 5 words in term frequency for each cluster 
top_words(mgp.cluster_word_distribution, top_index, 5)

Number of documents per topic : [ 0 33  0 31  0 13  2  0 83 24 15  0  0  0 23 17  0  8  0 16  0  0  0  0
  0  0 35  0 20  0  0  1  0 22  0  0 10  0  4  0  1 27  0  0  0  0 28  2
 30  0  0 31 64  0  0  1 41 36  4 38 14  5  0  4  4  0  0  0  3  0  0 89
  0  0  0  0 55  0  0 52 19  0  0  3  0  0  0  0 54  1  0 24  0  2  0  0
  0  0  0 11]
********************
Most important clusters (by number of docs inside): [71  8 52 76 88 79 56 59 57 26]
********************


NameError: name 'top_words' is not defined

In [266]:
mgp.cluster_word_distribution

[{'comprei ele, gastei com chip, recarga e nao funcionou. fica com a luz acesa e nao reconhece o chip. ligo e so da na caixa postal. nao funciona. decepcionado.': 1,
  'produto e bom, mas nao se engane achando que seja um ar condicionado': 1,
  'pela imagem ilustrativa achei que viria junto com o painel na embalagem, o suporte pra tv ate <NUM> polegadas! na minha opiniao em geral seria o correto esse item !': 1,
  'o produto nao reconheceu o chip. quanto as funcoes alguns funciona normalmente e outros nao.': 1,
  'embora tenha utilizado muito pouco, mas me parece ser um otimo produto. a entrega foi rapida. chegou tudo certinho. recomendo.': 1,
  'eu nao recebi meu produto.como voces querem que eu avalie minha compras.': 1,
  'excelente! eu recomendo!obrigado!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!': 1,
  'nao foi entregue minha mercadoria ,faz mais de trinta dias que estou esperando': 1,
  'a imagem para os canais e boa, mas para jogos e filmes on line e muito 

In [74]:
doc_count = np.array(mgp.cluster_doc_count)

In [75]:
# Show the top 5 words in term frequency for each cluster 
top_words(mgp.cluster_word_distribution, top_index, 5)

NameError: name 'top_words' is not defined

### Gibbs Sampling Dirichlet Mixture Model (GSDMM)